# Efficient Exploration of Phenol Derivatives Using Dynex

Molecule screening from a vast number of possible compounds is a challenging task. The emergence of quadratic unconstrained binary optimization (QUBO) solvers provides alternatives to address this issue. We propose a process for screening molecules by integrating QUBO solvers and density functional theory (DFT) calculations. As a proof-of-concept work, we map the problem of screening phenolic inhibitors onto the QUBO model. We approximate the bond dissociation energy (BDE) of the −OH bond, an indicator of good polymeric inhibitors, into the QUBO model by modifying the group contribution method (GCM) with the aid of DFT calculations. We demonstrate a strong correlation between this QUBO model and the data from DFT, with the correlation coefficient and Spearman’s coefficient of 0.82 and 0.86, respectively, when tested on the 85 given molecules. This mapping allows us to identify the candidates through the QUBO solver, whose BDEs are validated through DFT calculations, as well. Our work provides a promising direction for incorporating the GCM into QUBO solvers to tackle the molecule screening problems.

https://pubs.acs.org/doi/full/10.1021/acs.iecr.3c03331

In [44]:
from pyqubo import Binary, Constraint, SubH
from pyqubo import UnaryEncInteger,LogEncInteger
import numpy as np
import csv
import time

In [3]:
def csv_to_matrix(filename, size):
    num_rows, num_cols = size
    matrix = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            
            if len(row) != num_cols:
                raise ValueError("Number of columns in row does not match expected number of columns")
            matrix_row = []
            for value in row:
                try:
                    matrix_row.append(float(value))
                except ValueError:
                    matrix_row.append(value)
            matrix.append(matrix_row)
    if len(matrix) != num_rows:
        raise ValueError("Number of rows in matrix does not match expected number of rows")
    return matrix

## We input the weight coefficient calculated by DFT calculations from csv file

In [46]:
# In our proof-of concept work, there are 5 sites and 11 different molecular structure to choose.
num_of_position, num_of_mol = 5,11

# The contribution from one functional group.
w = csv_to_matrix('phenol-data/0117data_one_functional group.csv', (5,11))

# The contribution from the interactions of two functional groups
W10 = csv_to_matrix('phenol-data/0117data_R1R2_double_functional group.csv', (11,11))
W34 = csv_to_matrix('phenol-data/0117data_R1R2_double_functional group.csv', (11,11))
W21 = csv_to_matrix('phenol-data/0117data_R2R3_double_functional group.csv', (11,11))
W23 = csv_to_matrix('phenol-data/0117data_R2R3_double_functional group.csv', (11,11))

# second nearest neighbor interaction
W04 = csv_to_matrix('phenol-data/0117data_R1R5_double_functional group.csv', (11,11))
W20 = csv_to_matrix('phenol-data/0117data_R1R3_double_functional group.csv', (11,11))
W24 = csv_to_matrix('phenol-data/0117data_R1R3_double_functional group.csv', (11,11))
W31 = csv_to_matrix('phenol-data/0117data_R2R4_double_functional group.csv', (11,11))
W30 = csv_to_matrix('phenol-data/0117data_R1R4_double_functional group.csv', (11,11))
W14 = csv_to_matrix('phenol-data/0117data_R1R4_double_functional group.csv', (11,11))

### Here we label variables as $x_{ij}$, where $i$ labels different sites and $j$ labels different functional groups.

In [11]:
# create the binary variables
def create_x_vars(num_of_position, num_of_mol):
    a = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    for i in range(num_of_position):
        for j in range(num_of_mol):
            vars_name = 'x'+str(i)+'_'+str(j)
            a[i][j] = Binary(vars_name)
    return a

### We construct the objective function according to our weight coefficients.

In [12]:
def create_x_vars_name(num_of_position, num_of_mol):
    a0 = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    a1 = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    for i in range(num_of_position):
        for j in range(num_of_mol):
            vars_name = 'x'+str(i)+'_'+str(j)
            a1[i][j] = vars_name
    return a1

## We impose the constraints mentioned in the paper onto our objective function.

In [13]:
# construct objective function
x = create_x_vars(num_of_position, num_of_mol)
summ = 0
for i in range(num_of_position):
    for j in range(num_of_mol):
        summ += w[i][j]*x[i][j]
        

summ_10 = 0
summ_21 = 0
summ_23 = 0
summ_34 = 0
summ_04 = 0

summ_20 = 0
summ_24 = 0

summ_31 = 0
summ_30 = 0
summ_14 = 0


for i in range(1,num_of_mol):
    for j in range(1,num_of_mol):
        # nearest neighbor interaction
        summ_10 += W10[j][i]*x[1][j]*x[0][i]
        summ_34 += W34[j][i]*x[3][j]*x[4][i] 

        summ_21 += W21[j][i]*x[2][j]*x[1][i]  
        summ_23 += W23[j][i]*x[2][j]*x[3][i]

        summ_04 += W04[j][i]*x[0][j]*x[4][i] 

        # second nearest neighbor interaction
        summ_20 += W20[j][i]*x[2][j]*x[0][i] 
        summ_24 += W24[j][i]*x[2][j]*x[4][i]

        summ_31 += W31[j][i]*x[3][j]*x[1][i] 
        summ_30 += W30[j][i]*x[3][j]*x[0][i] 
        summ_14 += W14[j][i]*x[1][j]*x[4][i]

obj = (summ_10 + summ_34 + summ_21 + summ_23 + summ_04 + summ_20 + summ_24 + summ_31 + summ_30 + summ_14 ) + summ  + 87.5

In [15]:
H = obj
H_model = H.compile()
H_qubo, offset = H_model.to_qubo()
print(offset)

87.5


## We use Dynex to find the low energy solutions of the QUBO model.

In [43]:
import dynex
sampleset = dynex.sample_qubo(H_qubo, offset, mainnet=True, num_reads=50000, annealing_time = 2000);
print('Result:')
print(sampleset)

[DYNEX] PRECISION SET TO 0.001
[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬─────────────┬───────────┬───────────────────────────┬─────────┬─────────┬────────────────╮
│   DYNEXJOB │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CHIPS   │ STEPS   │ GROUND STATE   │
├────────────┼─────────────┼───────────┼───────────────────────────┼─────────┼─────────┼────────────────┤
│         -1 │           0 │           │ *** WAITING FOR READS *** │         │         │                │
╰────────────┴─────────────┴───────────┴───────────────────────────┴─────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.00 SECONDS
[DYNEX] SAMPLESET READY
Result:
  x0_1 x0_10 x0_2 x0_3 x0_4 x0_5 x0_6 x0_7 x0_8 ... x4_9     energy num_oc.
0    0     1    1    1    1    1    1    0    0 ...    0 -152.03753       1
['BINARY', 1 rows, 1 samples, 50 variables]


In [41]:
optimal = []
for i in samples.sample:
    if samples.sample[i]==1:
        optimal.append(i)

In [42]:
print('The lowest BDE is:',samples.energy+offset, 'kcal/mol')
print('with the structure:', optimal)

The lowest BDE is: -64.53753022400002 kcal/mol
with the structure: ['x0_10', 'x0_2', 'x0_3', 'x0_4', 'x0_5', 'x0_6', 'x1_10', 'x1_2', 'x1_5', 'x1_6', 'x1_7', 'x1_9', 'x2_1', 'x2_10', 'x2_2', 'x2_5', 'x2_7', 'x2_9', 'x3_10', 'x3_2', 'x3_5', 'x3_6', 'x3_7', 'x3_9', 'x4_10', 'x4_2', 'x4_3', 'x4_4', 'x4_5', 'x4_6']
